In [117]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [118]:
fund_name_dict = {
    0: 'Rural',
    1: 'Civil servants',
    2: 'Iranian + Self-employed',
    3: 'Universal',
    4: 'Foreign',
    5: 'Others'
}

fund_name_dict_reversed = {
    v: k for k, v in fund_name_dict.items()
}

## Importing

In [ ]:
costs_df = pd.read_pickle('../1403-4-16 export/yearly_costs_encr.pkl')
costs_df

In [ ]:
# data_df = costs_df['data']
data_df = pd.read_pickle('../edited_fund_person_data.pkl')
costs_df.drop('data', axis=1, level=0, inplace=True)
costs_df.fillna(0, inplace=True)
costs_df.rename({'pocket_adj': 'pocket', 'insurance_adj': 'insurance'},
             axis=1, level=1, inplace=True)
costs_df

In [121]:
# Setting outlier prescriptions to zero.
costs_df.loc[data_df['cat'] == 0, 'ab'] = 0

In [ ]:
costs_df.loc[data_df['cat'] == 0, 'ab']

In [123]:
model_df = costs_df.sum(axis=1).to_frame('total_cost')

In [ ]:
model_df

In [125]:
dominants_df = pd.read_pickle('../Drug class data/results/dominants_0.8.pkl')

In [126]:
data_df['dominant'] = dominants_df[0.8]

In [127]:
dominants_table = pd.read_excel('../Drug class data/results/IHIO_diabetes_antibiotic_2023_analysis_01/dominant_dm_class_table.xlsx',
                                header=[0, 1], index_col=0)
dominants_table

0                                 \
                                    drug proportion    ci_low     ci_up   
0                                  A10BA   0.428484  0.426747  0.430221   
1                            A10BA A10BB   0.207411  0.205988  0.208835   
2                                   A10A   0.108940  0.107846  0.110034   
3                                  A10BB   0.068220  0.067334  0.069105   
4                             A10A A10BA   0.037009  0.036346  0.037671   
..                                   ...        ...       ...       ...   
117         A10A A10BB A10BD A10BF A10BG   0.000003  0.000000  0.000009   
118  A10BA A10BB A10BD A10BF A10BG A10BX   0.000003  0.000000  0.000009   
119                    A10BA A10BB A10BJ   0.000003  0.000000  0.000009   
120        A10BA A10BD A10BF A10BG A10BX   0.000003  0.000000  0.000009   
121                          A10BB A10BJ   0.000003  0.000000  0.000009   

                                      1                                 \
    cumulative                     drug proportion    ci_low     ci_up   
0     0.428484                    A10BA   0.455529  0.453900  0.457158   
1     0.635895              A10BA A10BB   0.208192  0.206864  0.209520   
2     0.744835                     A10A   0.090730  0.089790  0.091669   
3     0.813055                    A10BB   0.060283  0.059505  0.061062   
4     0.850063               A10A A10BA   0.038541  0.037912  0.039171   
..         ...                      ...        ...       ...       ...   
117   0.999987  A10BA A10BF A10BJ A10BX   0.000003  0.000000  0.000008   
118   0.999990        A10BA A10BJ A10BX   0.000003  0.000000  0.000008   
119   0.999994  A10BB A10BD A10BG A10BX   0.000003  0.000000  0.000008   
120   0.999997              A10BB A10BJ   0.000003  0.000000  0.000008   
121   1.000000                      NaN        NaN       NaN       NaN   

                ...            3                                            \
    cumulative  ...         drug proportion    ci_low     ci_up cumulative   
0     0.455529  ...        A10BA   0.474123  0.472426  0.475820   0.474123   
1     0.663721  ...  A10BA A10BB   0.198660  0.197304  0.200016   0.672783   
2     0.754450  ...         A10A   0.078827  0.077911  0.079743   0.751610   
3     0.814734  ...        A10BB   0.048746  0.048014  0.049478   0.800357   
4     0.853275  ...   A10A A10BA   0.044681  0.043978  0.045383   0.845037   
..         ...  ...          ...        ...       ...       ...        ...   
117   0.999992  ...          NaN        NaN       NaN       NaN        NaN   
118   0.999994  ...          NaN        NaN       NaN       NaN        NaN   
119   0.999997  ...          NaN        NaN       NaN       NaN        NaN   
120   1.000000  ...          NaN        NaN       NaN       NaN        NaN   
121        NaN  ...          NaN        NaN       NaN       NaN        NaN   

               4                                            
            drug proportion    ci_low     ci_up cumulative  
0          A10BA   0.451134  0.449467  0.452802   0.451134  
1    A10BA A10BB   0.203072  0.201724  0.204420   0.654207  
2           A10A   0.079490  0.078583  0.080397   0.733696  
3     A10A A10BA   0.053715  0.052959  0.054470   0.787411  
4          A10BB   0.046742  0.046035  0.047450   0.834153  
..           ...        ...       ...       ...        ...  
117          NaN        NaN       NaN       NaN        NaN  
118          NaN        NaN       NaN       NaN        NaN  
119          NaN        NaN       NaN       NaN        NaN  
120          NaN        NaN       NaN       NaN        NaN  
121          NaN        NaN       NaN       NaN        NaN  

[122 rows x 25 columns]

In [128]:
# Selecting top 10 dominants of each group and naming the others as "other"
dominants_table.loc[0:9, pd.IndexSlice[:, 'drug']]

,0,1,2,3,4
,drug,drug,drug,drug,drug
0,A10BA,A10BA,A10BA,A10BA,A10BA
1,A10BA A10BB,A10BA A10BB,A10BA A10BB,A10BA A10BB,A10BA A10BB
2,A10A,A10A,A10A,A10A,A10A
3,A10BB,A10BB,A10BB,A10BB,A10A A10BA
4,A10A A10BA,A10A A10BA,A10A A10BA,A10A A10BA,A10BB
5,A10BA A10BB A10BG,A10A A10BA A10BB,A10A A10BA A10BB,A10A A10BA A10BB,A10A A10BA A10BB
6,A10A A10BA A10BB,A10BA A10BB A10BG,A10BA A10BB A10BF,A10BA A10BB A10BF,A10BA A10BB A10BF
7,A10BA A10BB A10BF,A10BA A10BB A10BF,A10BA A10BB A10BG,A10BA A10BB A10BG,A10BA A10BB A10BG
8,A10BG,A10BA A10BG,A10BA A10BG,A10BA A10BG,A10BA A10BG


In [129]:
dominants_10_first = set()
temp = dominants_table.loc[0:9, pd.IndexSlice[:, 'drug']]
for col in temp.columns:
    dominants_10_first.update(temp[col].values)

In [130]:
dominants_10_first

{'A10A',
 'A10A A10BA',
 'A10A A10BA A10BB',
 'A10BA',
 'A10BA A10BB',
 'A10BA A10BB A10BF',
 'A10BA A10BB A10BG',
 'A10BA A10BG',
 'A10BB',
 'A10BG'}

In [131]:
data_df['dom_10'] = data_df.loc[data_df['dominant'].isin(dominants_10_first), 'dominant']

In [132]:
data_df.drop('dominant', axis=1, inplace=True)

In [133]:
data_df['dom_10'].fillna('other', inplace=True)

/var/folders/lp/vx7z35tj789195nvj8ykll7h0000gn/T/ipykernel_16779/2089594623.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_df['dom_10'].fillna('other', inplace=True)


In [134]:
# Replacing A10BA to zero, so it would be the base.
data_df['dom_10'] = data_df['dom_10'].replace({'A10BA': 0})

In [135]:
data_df.replace({'fund': fund_name_dict_reversed}, inplace=True)

/var/folders/lp/vx7z35tj789195nvj8ykll7h0000gn/T/ipykernel_16779/2963476887.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_df.replace({'fund': fund_name_dict_reversed}, inplace=True)


In [ ]:
data_df

In [137]:
data_df['province'].value_counts()

province
23    293786
7     189214
9     187067
10    169262
2      99166
1      93854
11     67712
27     64084
8      63089
5      59901
12     48757
13     47700
15     44503
21     39769
0      34948
16     25690
22     25628
14     24355
18     23649
17     23122
28     21794
26     19195
19     19101
29     18836
Name: count, dtype: int64

In [138]:
# province_id
province_id = pd.read_excel('../diabeteschronicabuserprojectihio/Test_sample/Test/province_id.xlsx')
province_mapper = province_id.set_index('province_id_iso')['province_name_en']

In [139]:
# Replacing province names with their actual names and replacing tehran with 0_tehran to be the base.
province_mapper[23] = '0_' + province_mapper[23]
province_mapper
data_df['province'] = data_df['province'].replace(province_mapper)

In [ ]:
data_df

## Sandbox Model

In [141]:
test_model = smf.glm(
    'total_cost ~ age_cat',
    pd.concat([model_df, data_df['age_cat']], axis=1),
    family=sm.families.Gamma(link=sm.families.links.Log())
)

In [142]:
test_results = test_model.fit()

In [143]:
test_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             total_cost   No. Observations:              1704182
Model:                            GLM   Df Residuals:                  1704179
Model Family:                   Gamma   Df Model:                            2
Link Function:                    Log   Scale:                          16.016
Method:                          IRLS   Log-Likelihood:            -2.8366e+07
Date:                Mon, 21 Oct 2024   Deviance:                   3.8078e+06
Time:                        12:05:13   Pearson chi2:                 2.73e+07
No. Iterations:                     9   Pseudo R-squ. (CS):           0.004469
Covariance Type:            nonrobust                                         
==============================================================================================================
                                                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
Intercept                                     14.2561      0.006   2372.188      0.000      14.244      14.268
age_cat[T.Interval(40, 64, closed='both')]     0.5727      0.007     77.757      0.000       0.558       0.587
age_cat[T.Interval(65, 95, closed='both')]     0.7370      0.009     83.170      0.000       0.720       0.754
==============================================================================================================
"""

In [144]:
test_results.conf_int()

,0,1
Intercept,14.244343,14.267901
"age_cat[T.Interval(40, 64, closed='both')]",0.558290,0.587163
"age_cat[T.Interval(65, 95, closed='both')]",0.719666,0.754403


## Main

In [145]:
def run_gamma(formula, data):

    record_dict = dict()

    # Build model
    model = smf.glm(
        formula=formula,
        data=data,
        family=sm.families.Gamma(link=sm.families.links.Log())
    )
    
    # Fit model
    results = model.fit()

    # Record results
    record_dict['params'] = results.params
    record_dict['pvalues'] = results.pvalues
    record_dict['conf_int'] = results.conf_int()

    # Output
    return record_dict

In [146]:
# Dictionary for recording
record_dict = dict()

# Fitting models for one variable at a time.
for variable in data_df.columns:
    formula = 'total_cost' + ' ~ ' + f'C({variable})'
    record_dict[variable] = run_gamma(
        formula=formula,
        data=pd.concat([model_df, data_df[variable]], axis=1)
    )


In [147]:
cols = pd.MultiIndex.from_tuples([(  'params', 0),
            ( 'pvalues', 1),
            ('conf_int', 0),
            ('conf_int', 1)],
           )

In [148]:
record_df = pd.DataFrame()
to_concat = [pd.concat(v, axis=1) for v in record_dict.values()]


single_model_df = pd.concat(to_concat, axis=0, keys=record_dict.keys())

In [149]:
single_model_df.drop('Intercept', axis=0, level=1, inplace=True)

In [150]:
single_model_df[('params', 'exp')] = np.exp(single_model_df[('params', 0)])

In [151]:
single_model_df = single_model_df[['params', 'pvalues', 'conf_int']]

In [152]:
single_model_df

params            \
                                                               0       exp   
gender   C(gender)[T.1]                                 0.145749  1.156906   
fund     C(fund)[T.1]                                   0.659284  1.933408   
         C(fund)[T.2]                                   0.607889  1.836551   
         C(fund)[T.3]                                  -0.386155  0.679665   
         C(fund)[T.4]                                  -0.514249  0.597950   
         C(fund)[T.5]                                   0.643099  1.902367   
age_cat  C(age_cat)[T.Interval(40, 64, closed='both')]  0.572727  1.773095   
         C(age_cat)[T.Interval(65, 95, closed='both')]  0.737035  2.089729   
cat      C(cat)[T.1]                                    0.305134  1.356807   
         C(cat)[T.2]                                    0.517965  1.678609   
         C(cat)[T.3]                                    0.725952  2.066697   
         C(cat)[T.4]                                    1.124572  3.078899   
province C(province)[T.Bushehr]                        -0.202613  0.816594   
         C(province)[T.Chaharmahal and Bakhtiari]      -0.191390  0.825810   
         C(province)[T.Fars]                           -0.182163  0.833465   
         C(province)[T.Gilan]                           0.067469  1.069797   
         C(province)[T.Golestan]                       -0.098372  0.906311   
         C(province)[T.Hamadan]                        -0.221336  0.801447   
         C(province)[T.Hormozgan]                      -0.321925  0.724753   
         C(province)[T.Ilam]                           -0.299617  0.741102   
         C(province)[T.Isfahan]                         0.004781  1.004792   
         C(province)[T.Kerman]                          0.015837  1.015963   
         C(province)[T.Kermanshah]                     -0.391403  0.676107   
         C(province)[T.Khorasan, North]                -0.407636  0.665221   
         C(province)[T.Khorasan, Razavi]               -0.188714  0.828024   
         C(province)[T.Khorasan, South]                -0.290072  0.748209   
         C(province)[T.Kohgiluyeh and Boyer-Ahmad]     -0.351340  0.703744   
         C(province)[T.Kurdistan]                      -0.448595  0.638525   
         C(province)[T.Lorestan]                       -0.488148  0.613762   
         C(province)[T.Markazi]                        -0.239882  0.786721   
         C(province)[T.Mazandaran]                     -0.126203  0.881436   
         C(province)[T.Qazvin]                         -0.162321  0.850169   
         C(province)[T.Sistan and Baluchestan]         -0.439735  0.644207   
         C(province)[T.Yazd]                            0.155710  1.168487   
         C(province)[T.Zanjan]                          0.048579  1.049778   
dom_10   C(dom_10)[T.A10A]                              1.619155  5.048823   
         C(dom_10)[T.A10A A10BA]                        1.830853  6.239206   
         C(dom_10)[T.A10A A10BA A10BB]                  1.478730  4.387372   
         C(dom_10)[T.A10BA A10BB]                       0.192414  1.212173   
         C(dom_10)[T.A10BA A10BB A10BF]                 0.680796  1.975450   
         C(dom_10)[T.A10BA A10BB A10BG]                 0.543208  1.721521   
         C(dom_10)[T.A10BA A10BG]                       0.430142  1.537476   
         C(dom_10)[T.A10BB]                            -0.015392  0.984726   
         C(dom_10)[T.A10BG]                             0.261630  1.299045   
         C(dom_10)[T.other]                             0.896872  2.451921   

                                                              pvalues  \
                                                                    1   
gender   C(gender)[T.1]                                 2.955676e-116   
fund     C(fund)[T.1]                                    0.000000e+00   
         C(fund)[T.2]                                    0.000000e+00   
         C(fund)[T.3]     

In [153]:
import re


row_names = single_model_df.index.get_level_values(1)

pattern = r'C\((.*?)\)\[T\.(.*?)\]'

# Extract data
extracted_data = []
for name in row_names:
    match = re.search(pattern, name)
    if match:
        category = match.group(1)
        value = match.group(2)
        extracted_data.append([category, value])

# Create a DataFrame from the extracted data
df = pd.DataFrame(extracted_data, columns=['Category', 'Value'])

# Output the DataFrame
df

,Category,Value
0,gender,1
1,fund,1
2,fund,2
3,fund,3
4,fund,4
5,fund,5
6,age_cat,"Interval(40, 64, closed='both')"
7,age_cat,"Interval(65, 95, closed='both')"
8,cat,1
9,cat,2


In [154]:
single_model_df[['category', 'value']] = df.values

In [155]:
# Saving single_model_df
# single_model_df.to_excel('./results/GLM_single_variable_f.xlsx')

In [156]:
# Modeling all variables together
formula = ('total_cost' + ' ~ '
           + ' + '.join([f'C({variable})' for variable in data_df.columns])
)

record_dict = run_gamma(
    formula=formula,
    data=pd.concat([model_df, data_df], axis=1)
)

In [157]:
all_model_df = pd.concat(record_dict, axis=1)

In [158]:
all_model_df.drop('Intercept', axis=0, inplace=True)

In [159]:
all_model_df[('params', 'exp')] = np.exp(all_model_df[('params', 0)])

In [160]:
all_model_df = all_model_df[['params', 'pvalues', 'conf_int']]

In [161]:
all_model_df

params            \
                                                      0       exp   
C(gender)[T.1]                                -0.024593  0.975707   
C(fund)[T.1]                                   0.412595  1.510733   
C(fund)[T.2]                                   0.411546  1.509149   
C(fund)[T.3]                                  -0.324831  0.722649   
C(fund)[T.4]                                  -0.465192  0.628015   
C(fund)[T.5]                                   0.346776  1.414500   
C(age_cat)[T.Interval(40, 64, closed='both')]  0.287182  1.332667   
C(age_cat)[T.Interval(65, 95, closed='both')]  0.406627  1.501744   
C(cat)[T.1]                                    0.327703  1.387777   
C(cat)[T.2]                                    0.535773  1.708769   
C(cat)[T.3]                                    0.750373  2.117789   
C(cat)[T.4]                                    1.154352  3.171967   
C(province)[T.Bushehr]                        -0.265671  0.766692   
C(province)[T.Chaharmahal and Bakhtiari]      -0.185871  0.830381   
C(province)[T.Fars]                           -0.078636  0.924376   
C(province)[T.Gilan]                          -0.007751  0.992279   
C(province)[T.Golestan]                       -0.285431  0.751690   
C(province)[T.Hamadan]                        -0.202189  0.816941   
C(province)[T.Hormozgan]                      -0.385058  0.680411   
C(province)[T.Ilam]                           -0.464675  0.628339   
C(province)[T.Isfahan]                        -0.115181  0.891205   
C(province)[T.Kerman]                         -0.113255  0.892923   
C(province)[T.Kermanshah]                     -0.288234  0.749586   
C(province)[T.Khorasan, North]                -0.397382  0.672077   
C(province)[T.Khorasan, Razavi]               -0.185922  0.830338   
C(province)[T.Khorasan, South]                -0.356603  0.700050   
C(province)[T.Kohgiluyeh and Boyer-Ahmad]     -0.462979  0.629406   
C(province)[T.Kurdistan]                      -0.344971  0.708241   
C(province)[T.Lorestan]                       -0.416122  0.659599   
C(province)[T.Markazi]                        -0.216278  0.805511   
C(province)[T.Mazandaran]                     -0.249595  0.779116   
C(province)[T.Qazvin]                         -0.166479  0.846641   
C(province)[T.Sistan and Baluchestan]         -0.475759  0.621413   
C(province)[T.Yazd]                           -0.106712  0.898784   
C(province)[T.Zanjan]                          0.073733  1.076520   
C(dom_10)[T.A10A]                              1.710540  5.531950   
C(dom_10)[T.A10A A10BA]                        1.748980  5.748738   
C(dom_10)[T.A10A A10BA A10BB]                  1.329410  3.778813   
C(dom_10)[T.A10BA A10BB]                       0.080512  1.083842   
C(dom_10)[T.A10BA A10BB A10BF]                 0.593676  1.810632   
C(dom_10)[T.A10BA A10BB A10BG]                 0.498115  1.645617   
C(dom_10)[T.A10BA A10BG]                       0.360587  1.434172   
C(dom_10)[T.A10BB]                            -0.085260  0.918274   
C(dom_10)[T.A10BG]                             0.233830  1.263429   
C(dom_10)[T.other]                             0.840810  2.318243   

                                                     pvalues  conf_int  \
                                                           1         0   
C(gender)[T.1]                                  2.601880e-03 -0.040600   
C(fund)[T.1]                                   1.867081e-281  0.390037   
C(fund)[T.2]                                   7.929531e-185  0.383726   
C(fund)[T.3]                                   3.637280e-118 -0.352380   
C(fund)[T.4]                                    4.742853e-06 -0.664446   
C(fund)[T.5]                                   4.434949e-120  0.317606   
C(age_cat)[T.Interval(40, 64, closed='both')]  4.231690e-192  0.268144   
C(age_cat)[T.Interval(65, 95, closed='both')]  6.894542e-257  0.383349   
C(cat)[T.1]                                    2.584874e-161  0.3

In [162]:
import re


row_names = all_model_df.index

pattern = r'C\((.*?)\)\[T\.(.*?)\]'

# Extract data
extracted_data = []
for name in row_names:
    match = re.search(pattern, name)
    if match:
        category = match.group(1)
        value = match.group(2)
        extracted_data.append([category, value])

# Create a DataFrame from the extracted data
df = pd.DataFrame(extracted_data, columns=['Category', 'Value'])

# Output the DataFrame
df

,Category,Value
0,gender,1
1,fund,1
2,fund,2
3,fund,3
4,fund,4
5,fund,5
6,age_cat,"Interval(40, 64, closed='both')"
7,age_cat,"Interval(65, 95, closed='both')"
8,cat,1
9,cat,2


In [163]:
all_model_df[['category', 'value']] = df.values

In [164]:
all_model_df

params            \
                                                      0       exp   
C(gender)[T.1]                                -0.024593  0.975707   
C(fund)[T.1]                                   0.412595  1.510733   
C(fund)[T.2]                                   0.411546  1.509149   
C(fund)[T.3]                                  -0.324831  0.722649   
C(fund)[T.4]                                  -0.465192  0.628015   
C(fund)[T.5]                                   0.346776  1.414500   
C(age_cat)[T.Interval(40, 64, closed='both')]  0.287182  1.332667   
C(age_cat)[T.Interval(65, 95, closed='both')]  0.406627  1.501744   
C(cat)[T.1]                                    0.327703  1.387777   
C(cat)[T.2]                                    0.535773  1.708769   
C(cat)[T.3]                                    0.750373  2.117789   
C(cat)[T.4]                                    1.154352  3.171967   
C(province)[T.Bushehr]                        -0.265671  0.766692   
C(province)[T.Chaharmahal and Bakhtiari]      -0.185871  0.830381   
C(province)[T.Fars]                           -0.078636  0.924376   
C(province)[T.Gilan]                          -0.007751  0.992279   
C(province)[T.Golestan]                       -0.285431  0.751690   
C(province)[T.Hamadan]                        -0.202189  0.816941   
C(province)[T.Hormozgan]                      -0.385058  0.680411   
C(province)[T.Ilam]                           -0.464675  0.628339   
C(province)[T.Isfahan]                        -0.115181  0.891205   
C(province)[T.Kerman]                         -0.113255  0.892923   
C(province)[T.Kermanshah]                     -0.288234  0.749586   
C(province)[T.Khorasan, North]                -0.397382  0.672077   
C(province)[T.Khorasan, Razavi]               -0.185922  0.830338   
C(province)[T.Khorasan, South]                -0.356603  0.700050   
C(province)[T.Kohgiluyeh and Boyer-Ahmad]     -0.462979  0.629406   
C(province)[T.Kurdistan]                      -0.344971  0.708241   
C(province)[T.Lorestan]                       -0.416122  0.659599   
C(province)[T.Markazi]                        -0.216278  0.805511   
C(province)[T.Mazandaran]                     -0.249595  0.779116   
C(province)[T.Qazvin]                         -0.166479  0.846641   
C(province)[T.Sistan and Baluchestan]         -0.475759  0.621413   
C(province)[T.Yazd]                           -0.106712  0.898784   
C(province)[T.Zanjan]                          0.073733  1.076520   
C(dom_10)[T.A10A]                              1.710540  5.531950   
C(dom_10)[T.A10A A10BA]                        1.748980  5.748738   
C(dom_10)[T.A10A A10BA A10BB]                  1.329410  3.778813   
C(dom_10)[T.A10BA A10BB]                       0.080512  1.083842   
C(dom_10)[T.A10BA A10BB A10BF]                 0.593676  1.810632   
C(dom_10)[T.A10BA A10BB A10BG]                 0.498115  1.645617   
C(dom_10)[T.A10BA A10BG]                       0.360587  1.434172   
C(dom_10)[T.A10BB]                            -0.085260  0.918274   
C(dom_10)[T.A10BG]                             0.233830  1.263429   
C(dom_10)[T.other]                             0.840810  2.318243   

                                                     pvalues  conf_int  \
                                                           1         0   
C(gender)[T.1]                                  2.601880e-03 -0.040600   
C(fund)[T.1]                                   1.867081e-281  0.390037   
C(fund)[T.2]                                   7.929531e-185  0.383726   
C(fund)[T.3]                                   3.637280e-118 -0.352380   
C(fund)[T.4]                                    4.742853e-06 -0.664446   
C(fund)[T.5]                                   4.434949e-120  0.317606   
C(age_cat)[T.Interval(40, 64, closed='both')]  4.231690e-192  0.268144   
C(age_cat)[T.Interval(65, 95, closed='both')]  6.894542e-257  0.383349   
C(cat)[T.1]                                    2.584874e-161  0.3

In [165]:
# all_model_df.set_index(['category', 'value'])

In [166]:
# Saving all_variables_model_df
# all_model_df.to_excel('./results/GLM_all_variables_f.xlsx')